In [1]:
import yaml
import pymongo
from urllib.parse import quote_plus as quote
import hashlib

In [2]:
# Подключимся к нашей коллекции

with open('../env/creds.yaml', 'r') as file:
    creds_dict = yaml.safe_load(file)
    
url = 'mongodb://{user}:{pw}@{hosts}/?{rs}&authSource={auth_src}&{am}&tls=true&tlsCAFile={cert_file}'.format(
    user=creds_dict['username'],
    pw=quote(creds_dict['password']),
    hosts=creds_dict['host'],
    rs='replicaSet=rs01',
    auth_src=creds_dict['database'],
    am='authMechanism=DEFAULT',
    cert_file='../env/root.crt'
    )

dbs = pymongo.MongoClient(url)

db = dbs[creds_dict['database']]

collection = db['initial_dataset']

In [3]:
# Выгрузим данные из коллекции
pipeline = [{ "$unwind" : "$data_result.boxes" },
            {"$project":{
                "mass": "$data_result.boxes.mass",
                "size": "$data_result.boxes.size",
                "stacking": "$data_result.boxes.stacking",
                "turnover": "$data_result.boxes.turnover",
                "stacking_limit": "$data_result.boxes.stacking_limit",
                "loading_size": "$data_result.cargo_space.loading_size",
                "density_percent": "$data_result.cargo_space.calculation_info.density_percent",
                "filling_space_percent": "$data_result.cargo_space.calculation_info.filling_space_percent",
                }},
            {"$group": {"_id": "$_id",
                        "loading_size": {"$first":"$loading_size"},
                        "density_percent": {"$first":"$density_percent"},
                        "filling_space_percent": {"$first":"$filling_space_percent"},
                        "boxes": {
                            "$push":  {
                                "mass": "$mass",
                                "size": "$size",
                                "stacking": "$stacking",
                                "turnover": "$turnover",
                                "stacking_limit": "$stacking_limit"                               
                            }
                            
                        }
            }
            }
            ]
result = collection.aggregate(pipeline)
#item = result.next()
#print(item)
dataset = [i for i in result]

In [4]:
# stacking_limit везде = 0 - бесполезное значение
stacking_limits = list()
for item in dataset:
    for box in item['boxes']:
        if box['stacking_limit'] not in stacking_limits:
            stacking_limits.append(box['stacking_limit'])
print(stacking_limits)

[0.0]


In [5]:
# Размеры, похоже, что заданы в см.
dataset[0]['boxes'][0]

{'mass': 48.0,
 'size': {'width': 720.0, 'height': 600.0, 'length': 1420.0},
 'stacking': True,
 'turnover': True,
 'stacking_limit': 0.0}

In [6]:
# Как сделать размеры более единообразными (процент от максимальной стороны контейнера)?
loading_size = dataset[0]['loading_size']
box_size = dataset[0]['boxes'][0]['size']
print(f"Исходные размеры контейнера: {loading_size}")
print(f"Исходные размеры 1-й коробки: {box_size}")
norm_base = max(loading_size['width'], loading_size['height'], loading_size['length'])
print(f"Максимальная сторона: {norm_base}")
loading_size = {k: round(v*100/norm_base,2) for k,v in loading_size.items()}
box_size = {k: round(v*100/norm_base,2) for k,v in box_size.items()}
print(f"Нормализованные размеры контейнера: {loading_size}")
print(f"Нормализованные размеры 1-й коробки: {box_size}")

Исходные размеры контейнера: {'width': 1000.0, 'height': 2330.0, 'length': 1300.0}
Исходные размеры 1-й коробки: {'width': 720.0, 'height': 600.0, 'length': 1420.0}
Максимальная сторона: 2330.0
Нормализованные размеры контейнера: {'width': 42.92, 'height': 100.0, 'length': 55.79}
Нормализованные размеры 1-й коробки: {'width': 30.9, 'height': 25.75, 'length': 60.94}


In [7]:
def as_is_hash(box): #box_i = dataset[0]['boxes'][i]
    if box['turnover']:
        size_lst = sorted([x for x in box['size_scale'].values()])
    else:
        size_lst = [box['size_scale']['height'],
                    min(box['size_scale']['width'], box['size_scale']['length']),
                    max(box['size_scale']['width'], box['size_scale']['length'])]
    hash_object = hashlib.md5(
        ('h'+str(size_lst[0])+\
         'w'+str(size_lst[1])+\
         'l'+str(size_lst[2])+\
         's'+str(box['stacking'])+\
         't'+str(box['turnover'])
        ).encode())
    return hash_object.hexdigest()

In [8]:
def hash_cont(loading_size):
    hash_object = hashlib.md5(
        ('h'+str(loading_size['height'])+\
         'w'+str(loading_size['width'])+\
         'l'+str(loading_size['length'])
        ).encode())
    return hash_object.hexdigest()

In [9]:
def scale_item(item):
    loading_size = item['loading_size']
    norm_base = max(loading_size['width'], loading_size['height'], loading_size['length'])
    item['loading_size_scale'] = {k: round(v*100/norm_base,2) for k,v in loading_size.items()}
    for box in item['boxes']:
        box_size = box['size']
        box['size_scale'] = {k: round(v*100/norm_base,2) for k,v in box_size.items()}
        box['hash'] = as_is_hash(box)
    item['cont_hash'] = hash_cont(loading_size)
    item['boxes_hash'] = ','.join(sorted([box['hash'] for box in item['boxes']]))
    return item

In [10]:
dataset_scaled = [scale_item(item) for item in dataset]

In [11]:
len(dataset_scaled) # total containers

628

In [12]:
len({x['cont_hash'] for x in dataset_scaled}) # unique containers

68

In [13]:
# Уникальных контейнеро-наполнений
len({x['cont_hash']+'_'+x['boxes_hash'] for x in dataset_scaled})

493

In [14]:
targets = dict()
for item in dataset_scaled:
    k = item['cont_hash']+'_'+item['boxes_hash'] 
    if k not in targets.keys():
        targets[k] = [item['filling_space_percent']]
    else:
        if item['filling_space_percent'] not in targets[k]:
            targets[k].append(item['filling_space_percent'])

In [15]:
# одинаковые контейнеры с наполнениями имеют одинаковый filling_space_percent
max([len(x) for x in targets.values()])


1